<a href="https://colab.research.google.com/github/yuvrajtu/deeplearning/blob/master/ZS_Challenge_Using_Nueral_Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
df=pd.read_csv("/content/drive/My Drive/zschallenge/data.csv")
df.drop(['remaining_min.1','power_of_shot.1','knockout_match.1','remaining_sec.1','distance_of_shot.1'],axis=1,inplace=True)

In [0]:
features=['location_x','location_y','remaining_min','power_of_shot','remaining_sec','distance_of_shot','area_of_shot','shot_basics','range_of_shot','type_of_shot','type_of_combined_shot']
df['location_x'].fillna(value=-245,inplace=True)
df['location_y'].fillna(value=-39.0,inplace=True)
df['power_of_shot'].fillna(value=7.0,inplace=True)
df['remaining_sec'].fillna(value=11.0,inplace=True)
df['remaining_min'].fillna(value=7.0,inplace=True)
df['distance_of_shot'].fillna(value=28.0,inplace=True)
df['area_of_shot'].fillna(value='Center(C)',inplace=True)
df['shot_basics'].fillna(value='Goal Line',inplace=True)
df['range_of_shot'].fillna(value='16-24 ft.',inplace=True)
df['type_of_shot'].fillna(value='shot-1',inplace=True)
df['type_of_combined_shot'].fillna(value='shot-2',inplace=True)
#handling categorical values
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X = LabelEncoder()
df['area_of_shot'] = labelencoder_X.fit_transform(df['area_of_shot'])
df['shot_basics'] = labelencoder_X.fit_transform(df['shot_basics'])
df['type_of_shot'] = labelencoder_X.fit_transform(df['type_of_shot'])
df['range_of_shot'] = labelencoder_X.fit_transform(df['range_of_shot'])
df['type_of_combined_shot'] = labelencoder_X.fit_transform(df['type_of_combined_shot'])

In [0]:
dataset=df[df.is_goal.notnull()]
submissionset=df[df.is_goal.isnull()]



In [6]:
from sklearn.model_selection import train_test_split
X=dataset[features]
X=(X-X.min())/(X.max()-X.min())
import torch
Y=dataset.is_goal
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.10, random_state = 100)
print(X_train.to_numpy())

[[0.23895582 0.31616766 0.27272727 ... 0.25       0.43859649 1.        ]
 [0.45582329 0.05628743 0.27272727 ... 1.         1.         0.66666667]
 [0.01004016 0.32814371 0.18181818 ... 0.25       1.         0.5       ]
 ...
 [0.86144578 0.08023952 0.18181818 ... 0.         0.56140351 1.        ]
 [0.60240964 0.0994012  0.         ... 1.         1.         0.5       ]
 [0.01807229 0.0742515  0.27272727 ... 0.25       1.         0.5       ]]


In [19]:
import tensorflow as tf
import numpy as np
model = tf.keras.Sequential([
    
  tf.keras.layers.Dense(units=11,input_shape=[11]),
  tf.keras.layers.Dense(11, activation=tf.nn.relu),
#   tf.keras.layers.Dense(7, activation=tf.nn.sigmoid),
    tf.keras.layers.Dense(11, activation=tf.nn.tanh),
    tf.keras.layers.Dense(3, activation=tf.nn.relu),
    tf.keras.layers.Dense(1)
])
model.compile(optimizer='adam', 
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(X_train.to_numpy(), y_train.to_numpy(), epochs=100)
y_pred = model.predict(X_test.to_numpy())
print(y_pred)
y_pred.shape

Epoch 1/20
19543/19543 [==============================] - 1s 73us/sample - loss: 0.8298 - acc: 0.5646
Epoch 2/20
19543/19543 [==============================] - 1s 57us/sample - loss: 0.6758 - acc: 0.5915
Epoch 3/20
19543/19543 [==============================] - 1s 56us/sample - loss: 0.6736 - acc: 0.5954
Epoch 4/20
19543/19543 [==============================] - 1s 56us/sample - loss: 0.6720 - acc: 0.5966
Epoch 5/20
19543/19543 [==============================] - 1s 56us/sample - loss: 0.6707 - acc: 0.5971
Epoch 6/20
19543/19543 [==============================] - 1s 57us/sample - loss: 0.6700 - acc: 0.5973
Epoch 7/20
19543/19543 [==============================] - 1s 54us/sample - loss: 0.6691 - acc: 0.5978
Epoch 8/20
19543/19543 [==============================] - 1s 55us/sample - loss: 0.6684 - acc: 0.5985
Epoch 9/20
19543/19543 [==============================] - 1s 54us/sample - loss: 0.6679 - acc: 0.5969
Epoch 10/20
19543/19543 [==============================] - 1s 56us/sample - loss: 

(4886, 1)

In [20]:
Y=submissionset[features]
Y=(Y-Y.min())/(Y.max()-Y.min())
submissionset['is_goal']=model.predict(Y.to_numpy())
submissionset=submissionset.dropna(how='any')
submissionset[['shot_id_number','is_goal']].to_csv('/content/drive/My Drive/zschallenge/submission6.csv',index=False)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
